In [ ]:
# Import modules
import sys
import warnings
import shutil
warnings.filterwarnings('ignore')

import numpy as np
%matplotlib widget
from matplotlib import pyplot
import cmocean

import eatpy

sys.path.append("..")
import shared

# Experiment configuration
N = 20                     # ensemble size
varname = "P1_Chl"
parname = "instances_P1_parameters_sum"
cmap = cmocean.cm.algae

In [ ]:
# Read satellite observations of surface chlorophyll
obs = shared.read_0d_observations("P1_Chl_cci_5d.dat")

# Reference simulation

Forecast-only, no data assimilation

In [ ]:
# Set up the initial state, which is the result of a 7-year spin-up
shutil.copyfile('restart_01112014.nc', 'restart.nc')

# Run the model
!eat-gotm

## Load and plot results

In [ ]:
# Load results from GOTM's NetCDF output
time, z, ref, long_name, units = shared.read_result("result.nc", varname)

fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(
    figsize=(8, 6), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex="col"
)

# Plot time series of surface chlorophyll
shared.plot_0d_timeseries(ax1, time, ref[:, -1], obs)
ax1.set_ylabel(f"{long_name} ({units})")
ax1.set_title(f"surface {long_name}")
cax1.axis("off")

# Plot time-varying depth distributon of chlorophyll
pc, cb = shared.plot_1d_timeseries(ax2, time, z, ref, 20, cax=cax2, cmap=cmap)
cb.set_label(f"{long_name} ({units})")
ax2.set_title(long_name)

fig.tight_layout()

# Data assimilation

## Create the ensemble

In [ ]:
gotm = eatpy.models.gotm.YAMLEnsemble("gotm.yaml", N)
fabm = eatpy.models.gotm.YAMLEnsemble("fabm.yaml", N)
with gotm, fabm:
    gotm["fabm/yaml_file"] = fabm.file_paths
    fabm["instances/P1/parameters/sum"] *= np.random.uniform(low=0.7, high=1.3, size=N)

## Run data assimilation experiment

In [ ]:
# Set up the initial state, which is the result of a 7-year spin-up
# Note that this needs to be done again after the free run,
# as this run has saved its final state in restart.nc.
shutil.copyfile('restart_01112014.nc', 'restart.nc')

# Run the experiment
!mpiexec -n 1 python run.py : -n {N} eat-gotm --separate_gotm_yaml

## Load ensemble results

In [ ]:
enstime, ensz, ens, long_name, units = shared.read_ensemble_result("result.nc", varname, N)

## Plot

### Chlorophyll

In [ ]:
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(
    figsize=(10, 10),
    nrows=3,
    ncols=2,
    sharex="col",
    width_ratios=[0.95, 0.05],
    height_ratios=[0.4, 0.3, 0.3],
)

# Plot time series of surface chlorophyll
shared.plot_0d_ensemble_timeseries(ax1, enstime, ens[:, :, -1], [("model, free run", ref[:, -1])], obs, label="with DA")
ax1.set_ylabel(f"{long_name} ({units})")
ax1.set_title("surface diatom chlorophyll")
ax1.set_ylim(0.0, 4.0)
cax1.axis("off")

# Plot time-varying depth distributon of chlorophyll for free run
chl_contours = np.linspace(0.0, 2.0, 11)
pc, cb = shared.plot_1d_timeseries(
    ax2, time, z, ref, chl_contours, cmap=cmap, extend="max", cax=cax2
)
cb.set_label(f"{long_name} ({units})")
ax2.set_title("simulated diatom chlorophyll (free run)")

# Plot time-varying depth distributon of chlorophyll for run with DA
_, cb = shared.plot_1d_ensemble_timeseries(
    ax3, enstime, ensz, ens, chl_contours, cmap=cmap, extend="max", cax=cax3
)
cb.set_label(f"{long_name} ({units})")
ax3.set_title("simulated diatom chlorophyll (DA)")

fig.tight_layout()

fig.savefig("chl.png", dpi=150)

### Time variation in diatom growth rate

In [ ]:
fig, ax = pyplot.subplots(figsize=(8, 4))
enstime, _, par_ens, long_name, units = shared.read_ensemble_result(
    "result.nc", parname, N
)
constant_par = np.full_like(par_ens[0], 1.375)
shared.plot_0d_ensemble_timeseries(
    ax, enstime, par_ens, [("no DA", constant_par)], filter_period=31
)
ax.set_ylabel(f"maximum specific productivity ({units})")
ax.set_title("diatom maximum specific productivity")
fig.tight_layout()
fig.savefig("par.png", dpi=150)